In [1]:
import os
import sys
import argparse
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc
import traceback
from statistics import median
from statistics import mean
from statistics import mode
from statistics import stdev
from scipy import stats
from scipy import signal
import portion as P
import math
import random
import itertools

# ******************************* User Settings *******************************
database = "/home/wmnlab/D/database/"
dates = [
        #  "2023-02-04",
        #  "2023-02-04#1",
        #  "2023-02-27",
         "2023-03-15"
         ]
devices = [
    # "sm00",
    # "sm01",
    # "sm02",
    # "sm03",
    # "sm04",
    # "sm05",
    # "sm06",
    # "sm07",
    # "sm08",
    "qc00",
    "qc01",
    "qc02",
    "qc03",
]
schemes = [
    # "qc00",
    # "qc01",
    "B1",
    "B3",
    "B7",
    "B8",
    # "All@qc01",
    # "All@qc02",
    # "All@qc03",
]
exps = {  # experiment_name: (number_of_experiment_rounds, list_of_experiment_round)
            # If the list is None, it will not list as directories.
            # If the list is empty, it will list all directories in the current directory by default.
            # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    # "_Bandswitch": (1, ["#01",]),
    "_Bandlock_Udp_B1_B3_B7_B8_RM500Q": (6, ["#01", "#02", "#03", "#04", "#05", "#06"]),
    # "_Bandlock_Udp_B3_B7_B8_RM500Q": (2, ["#01", "#02"]),
    # "_Bandlock_Udp_all_RM500Q": (1, ["#01",]),
    # "_Bandlock_Udp_all_RM500Q": (2, ["#01", "#02"]),
}

INF = 2147483647
LENGTH = 250
DATA_RATE = 1000e3  # bits-per-second
PKT_RATE = DATA_RATE / LENGTH / 8  # packets-per-second
print("packet_rate (pps):", PKT_RATE, "\n")
# *****************************************************************************

packet_rate (pps): 500.0 



In [2]:
def makedir(dirpath, mode=0):  # mode=1: show message, mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)

In [4]:
for date in dates:
    for expr, (times, traces) in exps.items():
        for trace in traces:
            target_dir = os.path.join(database, date, expr, "combo", trace)
            makedir(target_dir)
            for tag in ["dnlk", "uplk"]:
            # for tag in ["dnlk",]:
                print("------------------------------------------")
                print(date, expr, trace, tag)  
                print("------------------------------------------")
                t = TicToc()
                t.tic()
                dfs = []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    source_dir = os.path.join(database, date, expr, dev, trace, "data")
                    dfs.append(pd.read_csv(os.path.join(source_dir, f"udp_{tag}_loss_latency.csv")))
                ### TODO 1
                st, et = [], []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    st.append(dfs[i]['sequence.number'].array[0])
                    et.append(dfs[i]['sequence.number'].array[-1])
                st, et = max(st), min(et)
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    dfs[i] = dfs[i][(dfs[i]['sequence.number'] >= st) & (dfs[i]['sequence.number'] <= et)]
                    dfs[i].reset_index(drop=True, inplace=True)
                # df = dfs[0][['sequence.number', 'Timestamp', 'Timestamp_epoch']]
                df = dfs[0][['sequence.number', 'Timestamp']]
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    # dfs[i] = dfs[i][['lost', 'excl', 'latency']]
                    dfs[i] = dfs[i][['transmit.time','arrival.time','lost','excl','latency']]
                    dfs[i].rename(
                        columns={
                            'transmit.time': f'transmit.time.{scheme}',
                            'arrival.time': f'arrival.time.{scheme}',
                            'lost': f'lost.{scheme}',
                            'excl': f'excl.{scheme}',
                            'latency': f'latency.{scheme}',
                        }, inplace=True
                    )
                df = pd.concat([df, *dfs], axis=1)
                ### TODO 2
                xs = list(itertools.combinations(range(len(schemes)), 2))
                for x in xs:
                    # f'lost.{schemes[x[0]]}.{schemes[x[1]]}'
                    # f'lost.{schemes[x[0]]}'
                    # f'lost.{schemes[x[1]]}'
                    df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}'] = df[f'lost.{schemes[x[0]]}'] & df[f'lost.{schemes[x[1]]}']
                    df[f'excl.{schemes[x[0]]}.{schemes[x[1]]}'] = df[f'excl.{schemes[x[0]]}'] & df[f'excl.{schemes[x[1]]}']
                    df[f'latency.{schemes[x[0]]}.{schemes[x[1]]}'] = df[[f'latency.{schemes[x[0]]}', f'latency.{schemes[x[1]]}']].min(axis=1)
                fout1 = os.path.join(target_dir, f"udp_{tag}_combo_loss_latency.csv")
                # print("output >>>", fout1)
                df.to_csv(fout1, index=False)
                ### TODO 3
                # colnames = list(df.columns)[3:]
                colnames = []
                data = []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    _df = df[df[f'lost.{scheme}'] == False]
                    # print(round(sum(df[f'lost.{scheme}']) / (len(df)+1e-9) * 100, 3))
                    # print(round(sum(_df[f'excl.{scheme}']) / (len(_df)+1e-9) * 100, 3))
                    loss = sum(df[f'lost.{scheme}']) / (len(df)+1e-9) * 100
                    excl = sum(_df[f'excl.{scheme}']) / (len(_df)+1e-9) * 100
                    latn = round(mean(_df[f'latency.{scheme}']), 6)
                    mlatn = round(max(_df[f'latency.{scheme}']), 6)
                    data = [*data, *[loss, excl, latn, mlatn]]
                    colnames = [*colnames, *[f'lost.{scheme}', f'excl.{scheme}', f'latency.{scheme}', f'mlatency.{scheme}']]
                    print(loss, excl, latn, mlatn, sep='\t')
                for x in xs:
                    _df = df[df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}'] == False]
                    # print(round(sum(df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(df)+1e-9) * 100, 3))
                    # print(round(sum(_df[f'excl.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(_df)+1e-9) * 100, 3))
                    loss = sum(df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(df)+1e-9) * 100
                    excl = sum(_df[f'excl.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(_df)+1e-9) * 100
                    latn = round(mean(_df[f'latency.{schemes[x[0]]}.{schemes[x[1]]}']), 6)
                    mlatn = round(max(_df[f'latency.{schemes[x[0]]}.{schemes[x[1]]}']), 6)
                    data = [*data, *[loss, excl, latn, mlatn]]
                    colnames = [*colnames, *[f'lost.{schemes[x[0]]}.{schemes[x[1]]}', f'excl.{schemes[x[0]]}.{schemes[x[1]]}', f'latency.{schemes[x[0]]}.{schemes[x[1]]}', f'mlatency.{schemes[x[0]]}.{schemes[x[1]]}']]
                    print(loss, excl, latn, mlatn, sep='\t')
                fout2 = os.path.join(target_dir, f"udp_{tag}_combo_statistics.csv")
                # print("output >>>", fout2)
                with open(fout2, "w", newline='') as fp:
                    writer = csv.writer(fp)
                    writer.writerow(colnames)
                    writer.writerow(data)
                t.toc()
                ### TODO END

------------------------------------------
2023-02-27 _Bandswitch #01 dnlk
------------------------------------------
0.6571064988800118	0.07341153909538951	0.022197	0.157208
1.2286328761618341	2.0523484471768754	0.026388	0.416516
0.004465049822514236	0.09079340036168453	0.021083	0.242325
Elapsed time is 6.137439 seconds.
------------------------------------------
2023-02-27 _Bandswitch #01 uplk
------------------------------------------
0.20092724201314066	0.5540352106899673	-0.00084	0.570313
0.3452971862744343	14.588576165122024	0.051875	0.997885
0.0	0.1964621921906264	-0.002797	0.515977
Elapsed time is 5.951775 seconds.


In [86]:
import os
import sys
import argparse
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc
import traceback
from statistics import median
from statistics import mean
from statistics import mode
from statistics import stdev
from scipy import stats
from scipy import signal
import portion as P
import math
import random
import itertools

# ******************************* User Settings *******************************
database = "/home/wmnlab/D/database/"
dates = [
        #  "2023-02-04",
        #  "2023-02-04#1",
         "2023-02-04#2",
         ]
devices = [
    # "sm00",
    # "sm01",
    # "sm02",
    # "sm03",
    # "sm04",
    # "sm05",
    # "sm06",
    # "sm07",
    # "sm08",
    # "qc00",
    "qc01",
    "qc02",
    "qc03",
]
schemes = [
    # "B3",
    # "B7",
    # "B8",
    "All@qc01",
    "All@qc02",
    "All@qc03",
]
exps = {  # experiment_name: (number_of_experiment_rounds, list_of_experiment_round)
            # If the list is None, it will not list as directories.
            # If the list is empty, it will list all directories in the current directory by default.
            # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    # "_Bandlock_Udp_B3_B7_B8_RM500Q": (1, ["#01",]),
    # "_Bandlock_Udp_B3_B7_B8_RM500Q": (2, ["#01", "#02"]),
    # "_Bandlock_Udp_all_RM500Q": (1, ["#01",]),
    "_Bandlock_Udp_all_RM500Q": (2, ["#01", "#02"]),
}

INF = 2147483647
LENGTH = 250
DATA_RATE = 1000e3  # bits-per-second
PKT_RATE = DATA_RATE / LENGTH / 8  # packets-per-second
print("packet_rate (pps):", PKT_RATE, "\n")
# *****************************************************************************

packet_rate (pps): 500.0 



In [87]:
for date in dates:
    for expr, (times, traces) in exps.items():
        for trace in traces:
            target_dir = os.path.join(database, date, expr, "combo", trace)
            makedir(target_dir)
            for tag in ["dnlk", "uplk"]:
            # for tag in ["dnlk",]:
                print("------------------------------------------")
                print(date, expr, trace, tag)  
                print("------------------------------------------")
                t = TicToc()
                t.tic()
                dfs = []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    source_dir = os.path.join(database, date, expr, dev, trace, "data")
                    dfs.append(pd.read_csv(os.path.join(source_dir, f"udp_{tag}_loss_latency.csv")))
                ### TODO 1
                st, et = [], []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    st.append(dfs[i]['sequence.number'].array[0])
                    et.append(dfs[i]['sequence.number'].array[-1])
                st, et = max(st), min(et)
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    dfs[i] = dfs[i][(dfs[i]['sequence.number'] >= st) & (dfs[i]['sequence.number'] <= et)]
                    dfs[i].reset_index(drop=True, inplace=True)
                df = dfs[0][['sequence.number', 'Timestamp', 'Timestamp_epoch']]
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    # dfs[i] = dfs[i][['sequence.number', 'Timestamp', 'Timestamp_epoch', 'lost', 'excl', 'latency']]
                    dfs[i] = dfs[i][['lost', 'excl', 'latency']]
                    dfs[i].rename(
                        columns={
                            'lost': f'lost.{scheme}',
                            'excl': f'excl.{scheme}',
                            'latency': f'latency.{scheme}',
                        }, inplace=True
                    )
                df = pd.concat([df, *dfs], axis=1)
                ### TODO 2
                xs = list(itertools.combinations(range(len(schemes)), 2))
                for x in xs:
                    # f'lost.{schemes[x[0]]}.{schemes[x[1]]}'
                    # f'lost.{schemes[x[0]]}'
                    # f'lost.{schemes[x[1]]}'
                    df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}'] = df[f'lost.{schemes[x[0]]}'] & df[f'lost.{schemes[x[1]]}']
                    df[f'excl.{schemes[x[0]]}.{schemes[x[1]]}'] = df[f'excl.{schemes[x[0]]}'] & df[f'excl.{schemes[x[1]]}']
                    df[f'latency.{schemes[x[0]]}.{schemes[x[1]]}'] = df[[f'latency.{schemes[x[0]]}', f'latency.{schemes[x[1]]}']].min(axis=1)
                fout1 = os.path.join(target_dir, f"udp_{tag}_combo_loss_latency.csv")
                # print("output >>>", fout1)
                df.to_csv(fout1, index=False)
                ### TODO 3
                # colnames = list(df.columns)[3:]
                colnames = []
                data = []
                for i, (dev, scheme) in enumerate(zip(devices, schemes)):
                    _df = df[df[f'lost.{scheme}'] == False]
                    # print(round(sum(df[f'lost.{scheme}']) / (len(df)+1e-9) * 100, 3))
                    # print(round(sum(_df[f'excl.{scheme}']) / (len(_df)+1e-9) * 100, 3))
                    loss = sum(df[f'lost.{scheme}']) / (len(df)+1e-9) * 100
                    excl = sum(_df[f'excl.{scheme}']) / (len(_df)+1e-9) * 100
                    latn = round(mean(_df[f'latency.{scheme}']), 6)
                    mlatn = round(max(_df[f'latency.{scheme}']), 6)
                    data = [*data, *[loss, excl, latn, mlatn]]
                    colnames = [*colnames, *[f'lost.{scheme}', f'excl.{scheme}', f'latency.{scheme}', f'mlatency.{scheme}']]
                    print(loss, excl, latn, mlatn, sep='\t')
                for x in xs:
                    _df = df[df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}'] == False]
                    # print(round(sum(df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(df)+1e-9) * 100, 3))
                    # print(round(sum(_df[f'excl.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(_df)+1e-9) * 100, 3))
                    loss = sum(df[f'lost.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(df)+1e-9) * 100
                    excl = sum(_df[f'excl.{schemes[x[0]]}.{schemes[x[1]]}']) / (len(_df)+1e-9) * 100
                    latn = round(mean(_df[f'latency.{schemes[x[0]]}.{schemes[x[1]]}']), 6)
                    mlatn = round(max(_df[f'latency.{schemes[x[0]]}.{schemes[x[1]]}']), 6)
                    data = [*data, *[loss, excl, latn, mlatn]]
                    colnames = [*colnames, *[f'lost.{schemes[x[0]]}.{schemes[x[1]]}', f'excl.{schemes[x[0]]}.{schemes[x[1]]}', f'latency.{schemes[x[0]]}.{schemes[x[1]]}', f'mlatency.{schemes[x[0]]}.{schemes[x[1]]}']]
                    print(loss, excl, latn, mlatn, sep='\t')
                fout2 = os.path.join(target_dir, f"udp_{tag}_combo_statistics.csv")
                # print("output >>>", fout2)
                with open(fout2, "w", newline='') as fp:
                    writer = csv.writer(fp)
                    writer.writerow(colnames)
                    writer.writerow(data)
                t.toc()
                ### TODO END

------------------------------------------
2023-02-04#2 _Bandlock_Udp_all_RM500Q #01 dnlk
------------------------------------------
0.37705843123172283	0.3586233278037894	0.013376	0.435054
0.3371173534822012	0.18824775168944935	0.012229	0.246817
0.20740045877274552	0.1410022912872329	0.011328	0.301152
0.0	0.015756571956233315	0.010514	0.138296
0.0	0.0	0.010026	0.076778
0.009893661460890687	0.010261108566193274	0.009902	0.120918
Elapsed time is 4.481300 seconds.
------------------------------------------
2023-02-04#2 _Bandlock_Udp_all_RM500Q #01 uplk
------------------------------------------
0.07878285978116659	0.23470132422887696	0.012711	0.760283
0.017955163391986804	0.1806833716323435	0.011383	0.576842
0.039208213937603834	0.1579940981322966	0.009254	0.487024
0.0	0.0007328638119178286	0.009971	0.163837
0.0	0.0051300466834248	0.008282	0.161914
0.0	0.002198591435753486	0.008091	0.161914
Elapsed time is 4.463226 seconds.
------------------------------------------
2023-02-04#2 _Bandloc